# pvlib-Python Forecasting Example (Tucson, AZ)
### Install Standard Release 

In [1]:
!pip install --upgrade pip

     |████████████████████████████████| 1.4MB 3.8MB/s eta 0:00:01     |█████████▌                      | 430kB 3.8MB/s eta 0:00:01
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [2]:
# Package from the Python Package Index
!pip install pvlib
# Module netCDF4 was not found 
!pip install netCDF4
# Module siphon was not found
!pip install siphon
# Downgrade to the prior release 
!pip install cftime==1.0.4.2
# Learned this from Birk
# Website: https://github.com/pvlib/pvlib-python/issues/895

     |████████████████████████████████| 19.3 MB 31 kB/s s eta 0:00:01
     |████████████████████████████████| 4.1 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 328 kB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 483 kB/s eta 0:00:011
     |████████████████████████████████| 308 kB 3.7 MB/s eta 0:00:01
  Attempting uninstall: cftime
    Found existing installation: cftime 1.1.1.2
    Uninstalling cftime-1.1.1.2:
      Successfully uninstalled cftime-1.1.1.2


In [3]:
# Make basic imports and then set the location and time range data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pytz
import os
import inspect
import seaborn as sns; sns.set_color_codes()
import datetime
import warnings
import time

# Import pvlib forecast models
from pvlib.forecast import GFS, NAM, NDFD, HRRR, RAP
from pvlib import solarposition

# New Github addition
from netCDF4 import num2date
from requests.exceptions import HTTPError
from xml.etree.ElementTree import ParseError
from pvlib.location import Location
from pvlib.irradiance import liujordan, get_extra_radiation, disc
from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS


# Specify Location (Phoenix, AZ)
latitude, longitude, tz = 32.2, -110.9, 'US/Arizona'

# Specify time range
start = pd.Timestamp(datetime.date.today(), tz=tz)
end = start + pd.Timedelta(days=7)
irrad_vars = ['ghi','dni','dhi']

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pvlib/forecast.py:20: UserWarning: The forecast module algorithms and features are highly experimental. The API may change, the functionality may be consolidated into an io module, or the module may be separated into its own package.
  'The forecast module algorithms and features are highly experimental. '


In [4]:
pytz.all_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/

In [5]:
pytz.country_timezones('US')

['America/New_York',
 'America/Detroit',
 'America/Kentucky/Louisville',
 'America/Kentucky/Monticello',
 'America/Indiana/Indianapolis',
 'America/Indiana/Vincennes',
 'America/Indiana/Winamac',
 'America/Indiana/Marengo',
 'America/Indiana/Petersburg',
 'America/Indiana/Vevay',
 'America/Chicago',
 'America/Indiana/Tell_City',
 'America/Indiana/Knox',
 'America/Menominee',
 'America/North_Dakota/Center',
 'America/North_Dakota/New_Salem',
 'America/North_Dakota/Beulah',
 'America/Denver',
 'America/Boise',
 'America/Phoenix',
 'America/Los_Angeles',
 'America/Anchorage',
 'America/Juneau',
 'America/Sitka',
 'America/Metlakatla',
 'America/Yakutat',
 'America/Nome',
 'America/Adak',
 'Pacific/Honolulu']

In [6]:
start

Timestamp('2020-04-04 00:00:00-0700', tz='US/Arizona')

In [7]:
start.to_pydatetime()

datetime.datetime(2020, 4, 4, 0, 0, tzinfo=<DstTzInfo 'US/Arizona' MST-1 day, 17:00:00 STD>)

In [8]:
end.to_pydatetime()

datetime.datetime(2020, 4, 11, 0, 0, tzinfo=<DstTzInfo 'US/Arizona' MST-1 day, 17:00:00 STD>)

In [9]:
datetime.datetime.strptime(time.ctime(), "%a %b %d %H:%M:%S %Y")

datetime.datetime(2020, 4, 4, 21, 22, 6)

### Instantiate a GFS model object and get the forecast data from Unidata

In [10]:
# GFS model, defaults to 0.5 degree resolution
# 0.25 deg available 
model = GFS() 

# Retrive data.returns panda.DataFrame object
raw_data = model.get_data(latitude, longitude, start, end)
print(raw_data.head())

                           Temperature_surface  \
2020-04-04 09:00:00-07:00           285.378510   
2020-04-04 12:00:00-07:00           283.329712   
2020-04-04 15:00:00-07:00           292.612122   
2020-04-04 18:00:00-07:00           308.816986   
2020-04-04 21:00:00-07:00           311.508972   

                           Total_cloud_cover_boundary_layer_cloud_Mixed_intervals_Average  \
2020-04-04 09:00:00-07:00                                                0.0                
2020-04-04 12:00:00-07:00                                                0.0                
2020-04-04 15:00:00-07:00                                                0.0                
2020-04-04 18:00:00-07:00                                                0.0                
2020-04-04 21:00:00-07:00                                                0.0                

                           Total_cloud_cover_middle_cloud_Mixed_intervals_Average  \
2020-04-04 09:00:00-07:00                              

In [23]:
print(type(start))
print(type(end))
print(type(irrad_vars))
print(type(model))
print(type(datetime))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'list'>
<class 'pvlib.forecast.GFS'>
<class 'module'>


In [ ]:
pd.Timestamp('2017-01-01T12')

In [ ]:
pd.Timestamp(1513393355.5, unit='s')

In [ ]:
pd.Timestamp(1513393355, unit='s', tz='US/Pacific')

In [ ]:
pd.Timestamp(2017, 1, 1, 12)

In [ ]:
pd.Timestamp(year=2017, month=1, day=1, hour=12)

In [ ]:
print(datetime)

## PV Power Forecast

In [ ]:
from pvlib.pvsystem import PVSystem, retrieve_sam
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
from pvlib.tracking import SingleAxisTracker
from pvlib.modelchain import ModelChain

sandia_modules = retrieve_sam('sandiamod')
cec_inverters = retrieve_sam('cecinverter')
module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
inverter = cec_inverters['SMA_America__SC630CP_US__with_ABB_EcoDry_Ultra_transformer_']
temperature_model_parameters = TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

# model a big tracker for more fun
system = SingleAxisTracker(module_parameters=module, inverter_parameters=inverter, temperature_model_parameters=temperature_model_parameters, modules_per_string=15, strings_per_inverter=300)

# fx is a common abbreviation for forecast
fx_model = GFS()
fx_data = fx_model.get_processed_data(latitude, longitude, start, end)

# use a ModelChain object to calculate modeling intermediates
mc = ModelChain(system, fx_model.location)

# extract relevant data for model chain
mc.run_model(fx_data)

### Plot of modeling Intermediates and the Forecast Power 

In [ ]:
mc.total_irrad.plot();
plt.ylabel('Plane of array irradiance ($W/m^2$)');
plt.legend(loc='best');